In [1]:
#import required libraries
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords

In [17]:
#import dataset into pandas dataframe
df_spamcollection = pd.read_csv('datasets/spam.csv',sep='\t', names=['response','message'])
df_spamcollection[df_spamcollection['response']=='ham']['message'][13]

"I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times."

In [5]:
df_spamcollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
#view response using groupby and describe method
df_spamcollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [7]:
df_spamcollection['length'] = df_spamcollection['message'].apply(len)
df_spamcollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
#define a function to get rid fo stopwords present in the messages
def message_text_process(mess):
    #check characters to see if there are punctuations
    no_punctuation = [char for char in mess if char not in string.punctuation]
    #now for the sentence without punctuation
    no_punctuation = ''.join(no_punctuation)
    #now eliminate any stopwords
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english') ]

In [10]:
#verify that function is working
df_spamcollection.head(5).apply(message_text_process)

LookupError: ("\n**********************************************************************\n  Resource \x1b[93mstopwords\x1b[0m not found.\n  Please use the NLTK Downloader to obtain the resource:\n\n  \x1b[31m>>> import nltk\n  >>> nltk.download('stopwords')\n  \x1b[0m\n  Attempted to load \x1b[93mcorpora/stopwords\x1b[0m\n\n  Searched in:\n    - 'C:\\\\Users\\\\msdaw/nltk_data'\n    - 'C:\\\\Users\\\\msdaw\\\\Anaconda3\\\\nltk_data'\n    - 'C:\\\\Users\\\\msdaw\\\\Anaconda3\\\\share\\\\nltk_data'\n    - 'C:\\\\Users\\\\msdaw\\\\Anaconda3\\\\lib\\\\nltk_data'\n    - 'C:\\\\Users\\\\msdaw\\\\AppData\\\\Roaming\\\\nltk_data'\n    - 'C:\\\\nltk_data'\n    - 'D:\\\\nltk_data'\n    - 'E:\\\\nltk_data'\n**********************************************************************\n", 'occurred at index response')

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msdaw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
#verify that function is working
df_spamcollection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [16]:
#start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
#bag of words by applying the function & fit the data into it
bag_of_words = CountVectorizer(analyzer=message_text_process).fit(df_spamcollection['message'])

In [18]:
#print the length of the bag of words
print(len(bag_of_words.vocabulary_))

11425


In [19]:
#store bag of words for messages using transform method
message_bagofwords = bag_of_words.transform(df_spamcollection['message'])

In [21]:
#apply tfidf transformer and fit the bag of words into it
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)
message_tfidf = tfidf_transformer.transform(message_bagofwords)

In [23]:
#print the shape of tfidf
print(message_tfidf.shape)

(5572, 11425)


In [24]:
#choose the naive bayes model to detect the spam & fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf,df_spamcollection['response'])

In [28]:
#check the model for predicted & expected value say for message #2 and #5
message = df_spamcollection['message'][2]
print('Normal message : ',message)
bagofwords_for_message = bag_of_words.transform([message])
print('bagofwords_for_message : ',bagofwords_for_message)
tfidf = tfidf_transformer.transform(bagofwords_for_message)
print('tfidf : ',tfidf)

print('Predicted value : ',spam_detect_model.predict(tfidf)[0])
print('expected value : ',df_spamcollection['response'][2])


Normal message :  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
bagofwords_for_message :    (0, 73)	1
  (0, 423)	1
  (0, 430)	1
  (0, 443)	1
  (0, 871)	1
  (0, 1551)	1
  (0, 1833)	2
  (0, 1941)	1
  (0, 2804)	1
  (0, 3953)	1
  (0, 4731)	1
  (0, 5619)	1
  (0, 6331)	2
  (0, 6557)	1
  (0, 9159)	1
  (0, 9200)	1
  (0, 9252)	1
  (0, 10512)	1
  (0, 10686)	1
  (0, 11084)	1
  (0, 11123)	1
tfidf :    (0, 11123)	0.19104387220509106
  (0, 11084)	0.15898145347176754
  (0, 10686)	0.13995540820792943
  (0, 10512)	0.2118842499614227
  (0, 9252)	0.16121646071144807
  (0, 9200)	0.22538631044314436
  (0, 9159)	0.22538631044314436
  (0, 6557)	0.18466951562322612
  (0, 6331)	0.347904654589528
  (0, 5619)	0.1887440015250524
  (0, 4731)	0.16452475271486866
  (0, 3953)	0.1544015632869991
  (0, 2804)	0.17524194104333074
  (0, 1941)	0.1526581729225716
  (0, 1833)	0.4507726208862887
  (0, 1551)	0.21778234